# EDA Unificado (Practica 4)

**Fuente de datos:** `data/listings.csv.gz`  
**Filas/Columnas:** 27,051 x 79  
**Generado:** 2026-02-04 16:22

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 120)

## 1) Carga de datos

In [ ]:
DATA_CANDIDATES = [
    Path('data/listings.csv.gz'),
    Path('listings.csv.gz'),
    Path('listings.csv'),
]

data_path = None
for p in DATA_CANDIDATES:
    if p.exists():
        data_path = p
        break

if data_path is None:
    raise FileNotFoundError('No se encontro listings.csv.gz ni listings.csv')

compression = 'gzip' if data_path.suffix == '.gz' else None
df = pd.read_csv(data_path, compression=compression, low_memory=False)

print('Data path:', data_path)
print('Shape:', df.shape)

## 2) Calidad de datos (nulos y cardinalidad)

| index | pct_missing |
|---|---|
| license | 100.0000 |
| neighbourhood_group_cleansed | 100.0000 |
| calendar_updated | 100.0000 |
| neighbourhood | 49.2218 |
| neighborhood_overview | 49.2218 |
| host_neighbourhood | 45.3366 |
| host_about | 40.0318 |
| host_location | 21.5408 |
| host_response_time | 14.6205 |
| host_response_rate | 14.6205 |
| bed_per_person | 12.9607 |
| beds | 12.9607 |
| bathrooms | 12.9237 |
| price_clean | 12.8794 |
| price | 12.8794 |

In [ ]:
# === 2) Nulos y cardinalidad ===
def detect_drop_candidates(df, missing_threshold=60, high_card_threshold=200, long_text_len=80):
    rows = []
    n = len(df)
    for col in df.columns:
        s = df[col]
        n_missing = s.isna().sum()
        pct_missing = (n_missing / n) * 100
        n_unique = s.nunique(dropna=True)
        sample = s.dropna().astype(str).head(3).tolist()
        mean_len = None
        if s.dtype == 'object':
            mean_len = s.dropna().astype(str).str.len().mean()
        reasons = []
        if pct_missing > missing_threshold:
            reasons.append(f'missing>{missing_threshold}%')
        if n_unique > high_card_threshold:
            reasons.append('high_card')
        if s.dtype == 'object' and mean_len and mean_len > long_text_len:
            reasons.append('long_text')
        if s.dtype == 'object' and s.astype(str).str.contains('http', case=False, na=False).mean() > 0.2:
            reasons.append('url_like')
        rows.append({
            'col': col,
            'pct_missing': round(pct_missing, 2),
            'n_unique': n_unique,
            'mean_len': None if mean_len is None else round(mean_len, 1),
            'sample': sample,
            'reasons': ', '.join(reasons)
        })
    return pd.DataFrame(rows).sort_values(by=['pct_missing','n_unique'], ascending=False)

missing_report = detect_drop_candidates(df)
missing_report.head(20)

## 3) Target y transformación (`price_clean` vs `log_price_clean`)

- Missing en `price_clean`: **12.88%**
- Skew `price_clean`: **52.125**
- Skew `log_price_clean`: **0.841**

**Justificación:** `log_price_clean` reduce la asimetría y estabiliza la regresión.

In [ ]:
# === 3) Target y transformación ===
if 'price' in df.columns:
    df['price_clean'] = (
        df['price'].astype(str)
        .str.replace(r'[,$]', '', regex=True)
    )
    df['price_clean'] = pd.to_numeric(df['price_clean'], errors='coerce')
    df['log_price_clean'] = np.log1p(df['price_clean'])

if 'price_clean' in df.columns:
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    sns.histplot(df['price_clean'].dropna(), bins=60, kde=True, ax=ax[0], color='#4C72B0')
    ax[0].set_title('price_clean (crudo)')
    ax[0].set_xlabel('price_clean')

    sns.histplot(df['log_price_clean'].dropna(), bins=60, kde=True, ax=ax[1], color='#55A868')
    ax[1].set_title('log_price_clean')
    ax[1].set_xlabel('log_price_clean')

    plt.tight_layout()

## 4) Outliers

- Clip 1–99%: **[224.66, 9918.58]**
- IQR bounds: **[-809.00, 3063.00]**
- % debajo IQR: **0.00%**
- % arriba IQR: **6.54%**

In [ ]:
# === 4) Outliers ===
if 'price_clean' in df.columns:
    q01, q99 = df['price_clean'].quantile([0.01, 0.99])
    df['price_clean_clip'] = df['price_clean'].clip(q01, q99)

    q1 = df['price_clean'].quantile(0.25)
    q3 = df['price_clean'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df['price_clean_iqr'] = df['price_clean'].clip(lower, upper)

    fig, ax = plt.subplots(1, 3, figsize=(14, 4))
    sns.boxplot(y=df['price_clean'], ax=ax[0], showfliers=False)
    ax[0].set_title('price_clean')
    sns.boxplot(y=df['price_clean_clip'], ax=ax[1], showfliers=False)
    ax[1].set_title('price_clean_clip (1-99%)')
    sns.boxplot(y=df['price_clean_iqr'], ax=ax[2], showfliers=False)
    ax[2].set_title('price_clean_iqr (IQR)')
    plt.tight_layout()

## 5) Rates del host

In [ ]:
# === 5) Rates ===
for col in ['host_response_rate', 'host_acceptance_rate']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace('%', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

if 'host_response_rate' in df.columns:
    df[['host_response_rate','host_acceptance_rate']].describe()

## 6) Amenities

In [ ]:
# === 6) Amenities ===

def parse_amenities(text):
    if pd.isna(text):
        return []
    text = str(text).strip('{}')
    parts = [p.strip().strip('"').strip("'") for p in text.split(',')]
    return [p for p in parts if p]

if 'amenities' in df.columns:
    df['amenities_list'] = df['amenities'].apply(parse_amenities)
    df['amenities_count'] = df['amenities_list'].apply(len)

    key_amenities = ['Wifi', 'Air conditioning', 'Pool', 'Kitchen', 'Parking']
    for amenity in key_amenities:
        col = f"has_{amenity.lower().replace(' ', '_')}"
        df[col] = df['amenities_list'].apply(
            lambda lst: int(any(amenity.lower() in a.lower() for a in lst))
        )

    # gráfico simple
    rates = df[[f"has_{a.lower().replace(' ', '_')}" for a in key_amenities]].mean().sort_values()
    rates.plot(kind='barh', figsize=(6,4), title='Amenities clave (proporción)')
    plt.tight_layout()

## 7) Capacidad y layout

In [ ]:
# === 7) Capacidad y layout ===
for col in ['accommodates','bedrooms','beds']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df['total_capacity'] = df['accommodates'].fillna(0) + df['bedrooms'].fillna(0) + df['beds'].fillna(0)
df['bed_per_person'] = df['beds'] / df['accommodates'].replace(0, np.nan)
df['bedroom_per_person'] = df['bedrooms'] / df['accommodates'].replace(0, np.nan)
df['space_per_person'] = df['total_capacity'] / df['accommodates'].replace(0, np.nan)

df[['total_capacity','bed_per_person','bedroom_per_person','space_per_person']].describe()

## 8) Geografía

In [ ]:
# === 8) Geografia ===

def haversine(lat, lon, lat0, lon0):
    R = 6371.0
    lat1 = np.radians(lat)
    lon1 = np.radians(lon)
    lat2 = np.radians(lat0)
    lon2 = np.radians(lon0)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

if 'latitude' in df.columns and 'longitude' in df.columns:
    ZOCALO = (19.4326, -99.1332)
    AICM = (19.4361, -99.0719)
    df['dist_zocalo_km'] = haversine(df['latitude'], df['longitude'], *ZOCALO)
    df['dist_aicm_km'] = haversine(df['latitude'], df['longitude'], *AICM)
    df['distance_from_center_km'] = df['dist_zocalo_km']
    df['is_central_location'] = (df['distance_from_center_km'] < 5).astype(int)

    df[['dist_zocalo_km','dist_aicm_km']].describe()

## 9) Recencia de reviews

In [ ]:
# === 9) Recencia de reviews ===
for col in ['last_scraped','last_review','first_review','host_since']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

if 'last_scraped' in df.columns and 'last_review' in df.columns:
    df['days_since_last_review'] = (df['last_scraped'] - df['last_review']).dt.days

if 'last_scraped' in df.columns and 'host_since' in df.columns:
    df['host_tenure_days'] = (df['last_scraped'] - df['host_since']).dt.days

if 'days_since_last_review' in df.columns:
    bins = [-1, 30, 90, 180, 365, 99999]
    labels = ['<=30', '31-90', '91-180', '181-365', '>365']
    df['recency_group'] = pd.cut(df['days_since_last_review'], bins=bins, labels=labels)
    df['recency_group'].value_counts(dropna=False)

## 10) Actividad / disponibilidad

In [ ]:
# === 10) Actividad / disponibilidad ===
if 'availability_365' in df.columns:
    df['availability_rate'] = df['availability_365'] / 365
    df['scarcity_score'] = 1 - df['availability_rate']

if 'maximum_nights' in df.columns and 'minimum_nights' in df.columns:
    df['booking_flexibility'] = df['maximum_nights'] - df['minimum_nights']

if 'availability_rate' in df.columns:
    df[['availability_rate','scarcity_score','booking_flexibility']].describe()

## 11) Lujo (NLP simple)

In [ ]:
# === 11) NLP simple para lujo ===
luxury_keywords = [
    'luxury','lujo','premium','exclusivo','exclusive','elegante','elegant',
    'boutique','vista','panoramica','private','privado','spacious','amplio'
]

if 'description' in df.columns:
    desc = df['description'].fillna('').str.lower()
    df['luxury_keyword_count'] = desc.apply(lambda x: sum(1 for kw in luxury_keywords if kw in x))
    df['is_luxury_property'] = (df['luxury_keyword_count'] > 0).astype(int)
    df[['luxury_keyword_count','is_luxury_property']].describe()

## 14) Propuesta de columnas a remover (con justificación)

- **100% nulos:** neighbourhood_group_cleansed, calendar_updated, license
- **IDs / URLs:** id, listing_url, scrape_id, host_id, host_url, picture_url, host_thumbnail_url, host_picture_url
- **Texto largo / alta cardinalidad:** name, description, neighborhood_overview, host_about, host_location, host_name, host_verifications
- **Fechas crudas (ya derivadas):** last_scraped, calendar_last_scraped, first_review, last_review, host_since
- **Redundantes / list-like:** bathrooms_text, amenities, neighbourhood, amenities_list
- **Leakage / demanda:** price, price_clean, log_price_clean, availability_30, availability_60, availability_90, availability_365, availability_eoy, has_availability, number_of_reviews, number_of_reviews_ltm, number_of_reviews_l30d, number_of_reviews_ly, reviews_per_month, estimated_occupancy_l365d, estimated_revenue_l365d, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin ...

**Resultado de la remoción:**
- Columnas removidas: **50**
- Shape original: **27051 x 104**
- Shape final (model): **27051 x 54**

In [ ]:
# === 14) Remoción de columnas (propuesta) ===
all_missing_cols = ['neighbourhood_group_cleansed','calendar_updated','license']
id_url_cols = ['id','listing_url','scrape_id','host_id','host_url','picture_url','host_thumbnail_url','host_picture_url']
text_cols = ['name','description','neighborhood_overview','host_about','host_location','host_name','host_verifications']
raw_date_cols = ['last_scraped','calendar_last_scraped','first_review','last_review','host_since']
redundant_cols = ['bathrooms_text','amenities','neighbourhood','amenities_list']
leakage_strict = [
    'price','price_clean','log_price_clean',
    'availability_30','availability_60','availability_90','availability_365','availability_eoy','has_availability',
    'number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','number_of_reviews_ly','reviews_per_month',
    'estimated_occupancy_l365d','estimated_revenue_l365d',
    'review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
    'review_scores_communication','review_scores_location','review_scores_value'
]

present = lambda cols: [c for c in cols if c in df.columns]

DROP_COLS = list(dict.fromkeys(
    present(all_missing_cols) + present(id_url_cols) + present(text_cols) +
    present(raw_date_cols) + present(redundant_cols) + present(leakage_strict)
))

print('Columnas a remover:', len(DROP_COLS))
DROP_COLS

## 15) Columnas finales (después de features)

- `source`
- `host_response_time`
- `host_response_rate`
- `host_acceptance_rate`
- `host_is_superhost`
- `host_neighbourhood`
- `host_listings_count`
- `host_total_listings_count`
- `host_has_profile_pic`
- `host_identity_verified`
- `neighbourhood_cleansed`
- `latitude`
- `longitude`
- `property_type`
- `room_type`
- `accommodates`
- `bathrooms`
- `bedrooms`
- `beds`
- `minimum_nights`
- `maximum_nights`
- `minimum_minimum_nights`
- `maximum_minimum_nights`
- `minimum_maximum_nights`
- `maximum_maximum_nights`
- `minimum_nights_avg_ntm`
- `maximum_nights_avg_ntm`
- `instant_bookable`
- `calculated_host_listings_count`
- `calculated_host_listings_count_entire_homes`
- `calculated_host_listings_count_private_rooms`
- `calculated_host_listings_count_shared_rooms`
- `amenities_count`
- `has_wifi`
- `has_air_conditioning`
- `has_pool`
- `has_kitchen`
- `has_parking`
- `total_capacity`
- `bed_per_person`
- `bedroom_per_person`
- `space_per_person`
- `dist_zocalo_km`
- `dist_aicm_km`
- `distance_from_center_km`
- `is_central_location`
- `days_since_last_review`
- `host_tenure_days`
- `recency_group`
- `availability_rate`
- `scarcity_score`
- `booking_flexibility`
- `luxury_keyword_count`
- `is_luxury_property`

In [ ]:
# === 15) Dataset final ===
df_model = df.drop(columns=DROP_COLS, errors='ignore').copy()
# eliminar list-like si aparece
for c in df_model.columns:
    if df_model[c].apply(lambda v: isinstance(v, list)).any():
        df_model = df_model.drop(columns=[c])

print('Shape final:', df_model.shape)
df_model.head(3)

## 16) Modelado (Regresión sobre `log_price_clean`)
Se entrena con pipeline (imputación + escalado + one-hot). Métricas reportadas en escala log y también re-transformadas a escala original.

| index | model | rmse_log | mae_log | r2_log | rmse_orig | mae_orig | r2_orig |
|---|---|---|---|---|---|---|---|
| 0 | RandomForest | 0.3719 | 0.2497 | 0.7527 | 9336.5418 | 601.3609 | 0.4300 |
| 1 | ExtraTrees | 0.3732 | 0.2455 | 0.7510 | 2722.8286 | 441.9990 | 0.9515 |
| 2 | GradientBoosting | 0.4286 | 0.3098 | 0.6714 | 11449.3053 | 733.0579 | 0.1428 |
| 3 | MLP | 0.4645 | 0.3287 | 0.6141 | 10577.3905 | 701.4562 | 0.2684 |
| 4 | LinearRegression | 0.4705 | 0.3318 | 0.6041 | 78566.7108 | 1900.5180 | -39.3634 |
| 5 | Ridge | 0.4750 | 0.3360 | 0.5965 | 113461.1089 | 2412.8868 | -83.1791 |
| 6 | Lasso | 0.4862 | 0.3452 | 0.5772 | 129765.2434 | 2656.1749 | -109.1100 |
| 7 | Baseline (median) | 0.7478 | 0.5651 | -0.0001 | nan | nan | nan |

**Nota:** `r2_orig` puede ser inestable por la alta asimetría del precio. Se recomienda comparar principalmente `r2_log` y RMSE/MAE en escala original.

In [ ]:
# === 16) Modelado (Regresión) ===
# Ver resultados en la tabla de arriba (generados por corrida externa).
# Aquí dejamos el pipeline base para reproducir.
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor

# target
y = df['log_price_clean']
mask = y.notna()
X = df_model.loc[mask].copy()
y = y.loc[mask].copy()

num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(exclude=[np.number]).columns

try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', ohe)
])

preprocess = ColumnTransformer(
    transformers=[('num', numeric_transformer, num_cols),
                  ('cat', categorical_transformer, cat_cols)]
)

## 17) Clasificación (segmentos de precio: low/mid/high)
Se segmenta el precio con `qcut` y se evalúan clasificadores con `accuracy` y `F1_macro`.

| index | model | accuracy | f1_macro |
|---|---|---|---|
| 0 | RandomForest | 0.7724 | 0.7726 |
| 1 | GradientBoosting | 0.7259 | 0.7263 |
| 2 | LogisticRegression | 0.6996 | 0.7000 |
| 3 | KNN | 0.6843 | 0.6835 |

In [ ]:
# === 17) Clasificación por segmentos de precio ===
# Se deja el pipeline base para reproducir.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

# target: segmentos
mask = df['price_clean'].notna()
X = df_model.loc[mask].copy()
y = df.loc[mask, 'price_clean']

y_seg = pd.qcut(y, q=3, labels=['low','mid','high'])

num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(exclude=[np.number]).columns

try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

preprocess = ColumnTransformer(
    transformers=[('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('scaler', StandardScaler())]), num_cols),
                  ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                          ('onehot', ohe)]), cat_cols)]
)